In [46]:
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame as DF
import seaborn as sns
sns.set(color_codes=True)
#np.random.seed(sum(map(ord, "distributions")))
from sklearn import linear_model  # packages for the logistic regression function to plot the logistic regression 
from sklearn.linear_model import LogisticRegression # packages for the logistic regression function to plot the logistic regression 
import scipy
from scipy import stats, integrate
from scipy.stats import mode
from scipy.stats.stats import pearsonr # Pearson's correlation
from copy import copy as copy
import operator as operator
import pylab

# Import Fixation reports

In [97]:
path_fixat ="/Users/bdmlab/OneDrive - University College London/PhD Projects/FoodFramingValue/Reports/Fixations_FullReport_RTfix_v4.txt"
# load csv for IA and Sacc
fixat_data = [pd.read_csv(path_fixat, sep="\t")]
# Concatenate to generate frames
fixat_data = pd.concat(fixat_data)

path_mssg ="/Users/bdmlab/OneDrive - University College London/PhD Projects/FoodFramingValue/Reports/Messages_FullReport_v1.txt"
# load csv for IA and Sacc
mssg_data = [pd.read_csv(path_mssg, sep="\t")]
# Concatenate to generate frames
mssg_data = pd.concat(mssg_data)

In [98]:
mssg_data

,DATA_FILE,trial,CURRENT_MSG_TEXT,CURRENT_MSG_FIX_DURATION,CURRENT_MSG_INTEREST_AREAS,CURRENT_MSG_INTEREST_AREA_ID,CURRENT_MSG_TIME,CURRENT_MSG_FIX_START,CURRENT_MSG_FIX_END,CURRENT_MSG_INDEX,CHOICE_RT,Block_Cond,IP_START_TIME,IP_DURATION,IP_END_TIME,trial_type
0,p1.edf,145,DISPLAY_IMAGES_ONSET,311,.,.,1093,1093,1402,1,22580,1.0,4281406,22565,4303971,Practice
1,p1.edf,145,NONE_FT,311,.,.,1120,1093,1402,2,22580,1.0,4281406,22565,4303971,Practice
2,p1.edf,145,ALL_INVISBLE,311,.,.,1121,1093,1402,3,22580,1.0,4281406,22565,4303971,Practice
3,p1.edf,145,LOWERLEFT_FT,126,.,.,1510,1444,1568,4,22580,1.0,4281406,22565,4303971,Practice
4,p1.edf,145,LOWER_LEFT_VISIBLE,126,.,.,1511,1444,1568,5,22580,1.0,4281406,22565,4303971,Practice
5,p1.edf,145,DISPLAY_IMAGES_ONSET,126,.,.,1526,1444,1568,6,22580,1.0,4281406,22565,4303971,Practice
6,p1.edf,145,LOWERRIGHT_FT,386,.,.,2793,2726,3110,7,22580,1.0,4281406,22565,4303971,Practice
7,p1.edf,145,LOWER_RIGHT_VISIBLE,386,.,.,2793,2726,3110,8,22580,1.0,4281406,22565,4303971,Practice
8,p1.edf,145,DISPLAY_IMAGES_ONSET,386,.,.,2809,2726,3110,9,22580,1.0,4281406,22565,4303971,Practice
9,p1.edf,145,LOWERLEFT_FT,318,.,.,4092,4026,4342,10,22580,1.0,4281406,22565,4303971,Practice


In [99]:
participant = fixat_data['DATA_FILE'].unique()
participant_names = fixat_data['DATA_FILE'].unique()

#Extract just names of participant (for folder name in future steps)
yy=0
for xx in participant_names:
    participant_names[yy]=xx.rstrip(".edf")
    yy+=1
print (participant_names)

trial_n = fixat_data['trial'].unique()

print("Number of participants:", len(participant))
print ("Number of trials:", len(trial_n))

['p1' 'p5' 'p6' 'p7' 'p8' 'p9' 'p10' 'p11' 'p12' 'p13' 'p14' 'p15' 'p16'
 'p17' 'p18' 'p19' 'p20' 'p21' 'p22' 'p23' 'p24' 'p25' 'p26' 'p27' 'p28'
 'p29' 'p30' 'p31' 'p32' 'p33' 'p34' 'p35' 'p36' 'p37' 'p38' 'p39' 'p40'
 'p41' 'p42' 'p43']
Number of participants: 40
Number of trials: 240


In [100]:
participant

array(['p1.edf', 'p5.edf', 'p6.edf', 'p7.edf', 'p8.edf', 'p9.edf',
       'p10.edf', 'p11.edf', 'p12.edf', 'p13.edf', 'p14.edf', 'p15.edf',
       'p16.edf', 'p17.edf', 'p18.edf', 'p19.edf', 'p20.edf', 'p21.edf',
       'p22.edf', 'p23.edf', 'p24.edf', 'p25.edf', 'p26.edf', 'p27.edf',
       'p28.edf', 'p29.edf', 'p30.edf', 'p31.edf', 'p32.edf', 'p33.edf',
       'p34.edf', 'p35.edf', 'p36.edf', 'p37.edf', 'p38.edf', 'p39.edf',
       'p40.edf', 'p41.edf', 'p42.edf', 'p43.edf'], dtype=object)

In [101]:
# From BlinkMatrix (from FoodFrameAnalysis.ipnyb) we find that we have to take out participants
Part2Delete = ['p9.edf' , 'p10.edf' , 'p20.edf' , 'p28.edf', 'p33.edf' , 'p36.edf', 'p40.edf']
for i in Part2Delete:
    print(i)
    fixat_data = fixat_data[(fixat_data.DATA_FILE != i)]
    mssg_data = mssg_data[(mssg_data.DATA_FILE != i)]

participant =  fixat_data["DATA_FILE"].unique()
print(fixat_data["DATA_FILE"].unique())
len(fixat_data["DATA_FILE"].unique())


p9.edf
p10.edf
p20.edf
p28.edf
p33.edf
p36.edf
p40.edf
['p1.edf' 'p5.edf' 'p6.edf' 'p7.edf' 'p8.edf' 'p11.edf' 'p12.edf'
 'p13.edf' 'p14.edf' 'p15.edf' 'p16.edf' 'p17.edf' 'p18.edf' 'p19.edf'
 'p21.edf' 'p22.edf' 'p23.edf' 'p24.edf' 'p25.edf' 'p26.edf' 'p27.edf'
 'p29.edf' 'p30.edf' 'p31.edf' 'p32.edf' 'p34.edf' 'p35.edf' 'p37.edf'
 'p38.edf' 'p39.edf' 'p41.edf' 'p42.edf' 'p43.edf']


33

In [102]:
# I used 0 for blank fixations, 1 for left, 2 for right, 3 for center.
participant[1]

'p5.edf'

In [103]:
#i=0
#j=1
#fixat_data_trial=fixat_data.loc[(fixat_data['DATA_FILE']==participant[i]) & (fixat_data['trial']== j) & (fixat_data['trial_type']== 'Experiment')]
#mssg_data_trial=mssg_data.loc[(mssg_data['DATA_FILE']==participant[i]) & (mssg_data['trial']== j) & (mssg_data['trial_type']== 'Experiment')]
#len(mssg_data_trial)

In [107]:
print(fixat_data_trial.iloc[0])

fixat_data_trial.iloc[0]


#fixat_data_trial.iloc[4].CURRENT_FIX_INTEREST_AREA_ID == '1'

DATA_FILE                                        p1.edf
trial                                                 1
CURRENT_FIX_INTEREST_AREA_ID                          .
CURRENT_FIX_DURATION                                240
CURRENT_FIX_START                                  1290
CURRENT_FIX_END                                    1528
CURRENT_FIX_INTEREST_AREA_DWELL_TIME                  .
CHOICE_RT                                          3327
CURRENT_FIX_BLINK_AROUND                         BEFORE
CURRENT_FIX_INTEREST_AREAS                          [ ]
CURRENT_FIX_PUPIL                                  4531
CURRENT_FIX_RUN_DWELL_TIME                            .
CURRENT_FIX_NEAREST_INTEREST_AREA_LABEL    IA_LEFT_ITEM
CURRENT_FIX_NEAREST_INTEREST_AREA                     1
CURRENT_FIX_INDEX                                     1
Block_Cond                                            2
trial_type                                   Experiment
IP_START_TIME                                   

DATA_FILE                                        p1.edf
trial                                                 1
CURRENT_FIX_INTEREST_AREA_ID                          .
CURRENT_FIX_DURATION                                240
CURRENT_FIX_START                                  1290
CURRENT_FIX_END                                    1528
CURRENT_FIX_INTEREST_AREA_DWELL_TIME                  .
CHOICE_RT                                          3327
CURRENT_FIX_BLINK_AROUND                         BEFORE
CURRENT_FIX_INTEREST_AREAS                          [ ]
CURRENT_FIX_PUPIL                                  4531
CURRENT_FIX_RUN_DWELL_TIME                            .
CURRENT_FIX_NEAREST_INTEREST_AREA_LABEL    IA_LEFT_ITEM
CURRENT_FIX_NEAREST_INTEREST_AREA                     1
CURRENT_FIX_INDEX                                     1
Block_Cond                                            2
trial_type                                   Experiment
IP_START_TIME                                   

In [109]:
i=0
fixations = []
#Run for each individual participant
for i in range(len(participant)):
    trial_n = 240
    for  j in range(trial_n):
        fixat_data_trial=fixat_data.loc[(fixat_data['DATA_FILE']==participant[i]) & (fixat_data['trial']== j+1) & (fixat_data['trial_type']== 'Experiment')]
        mssg_data_trial=mssg_data.loc[(mssg_data['DATA_FILE']==participant[i]) & (mssg_data['trial']== j+1) & (mssg_data['trial_type']== 'Experiment')]

        fixat_n = len(fixat_data_trial)
                
        current_trial_time = 0

        trial_start_time = mssg_data_trial.iloc[0].CURRENT_MSG_TIME
        
        #if first fixation is at the center or outside the area of the items we consider the trial start with the blank fixation
        if fixat_data_trial.iloc[0].CURRENT_FIX_INTEREST_AREA_ID == '.' or fixat_data_trial.iloc[0].CURRENT_FIX_INTEREST_AREA_ID == '4':
            trial_fix_init = mssg_data_trial.iloc[0].CURRENT_MSG_TIME
        #if first fixation is over one of the items we consider the trial start with that item
        if fixat_data_trial.iloc[0].CURRENT_FIX_INTEREST_AREA_ID == '1' or fixat_data_trial.iloc[0].CURRENT_FIX_INTEREST_AREA_ID == '2':
            #we assume center fixation since the beginning of the trial
            parcode_fix = i
            trial_fix = fixat_data_trial.iloc[0].trial - 1
            fix_time_fix = fixat_data_trial.iloc[0].CURRENT_FIX_START - mssg_data_trial.iloc[0].CURRENT_MSG_TIME
            block_fix =fixat_data_trial.iloc[0].Block_Cond
            fix_item_fix =  3
            
            #Extract reaction time for that trial (only for test)
            RT_fix_test = fixat_instance_data.CHOICE_RT 
            part_name_fix_test = fixat_instance_data.DATA_FILE 
            
            
            fixations.append([part_name_fix_test,parcode_fix, trial_fix, fix_item_fix, fix_time_fix,RT_fix_test,block_fix]) 
            #and we reset the fixation for the trial
            trial_fix_init = fixat_data_trial.iloc[0].CURRENT_FIX_START
        
        
        for  k in range(fixat_n):

            # extract saccade data for this participant
            fixat_instance_data = fixat_data_trial.iloc[k]
            parcode_fix = i
            trial_fix = fixat_instance_data.trial - 1 # remember in the aDDM_toolbox format it starts from zero. 
            block_fix =fixat_instance_data.Block_Cond


            #Considered the fixation duration as the end of the previous fixations and 
            #the end of current fixation. In case of the first fixation, we consider,
            # the beginning of the trial as the start, in case we start in blank
            
                        
            #Include condition to define fix_time_fix considering that we have the last fixation
            if k == fixat_n-1:  # if this is the last element
                #calculate the expected time of trial ending considering RT as the real length 
                RT_final_time = fixat_instance_data.CHOICE_RT + trial_start_time 
                fix_time_fix = RT_final_time - trial_fix_init

            # if not, calculate time for fixation as the time between the end of  previous fixation
            # and the end of current fixatoon
            else:
                fix_time_fix = fixat_instance_data.CURRENT_FIX_END - trial_fix_init
            
            if fixat_instance_data.CURRENT_FIX_INTEREST_AREA_ID == '1': 
                fix_item_fix =  1
            elif fixat_instance_data.CURRENT_FIX_INTEREST_AREA_ID == '2': 
                fix_item_fix =  2
            elif fixat_instance_data.CURRENT_FIX_INTEREST_AREA_ID == '.': 
                fix_item_fix =  0
            elif fixat_instance_data.CURRENT_FIX_INTEREST_AREA_ID == '4' and k==0: 
            # we assign center fixation only if it's the first one 
                fix_item_fix =  3    
            else  :
                fix_item_fix =  0   
                
            #Extract reaction time for that trial (only for test)
            RT_fix_test = fixat_instance_data.CHOICE_RT 
            part_name_fix_test = fixat_instance_data.DATA_FILE 



            
            
            fixations.append([part_name_fix_test,parcode_fix, trial_fix, fix_item_fix, fix_time_fix,RT_fix_test,block_fix]) 
            
            #update the init to the end of the current fixation
            trial_fix_init = fixat_instance_data.CURRENT_FIX_END


In [113]:
fixation_file = pd.DataFrame(fixations,columns=['name','parcode', 'trial', 'fix_item', 'fix_time','rt','block'])



In [115]:
fixation_frame_like = fixation_file.loc[fixation_file['block'] == 1] # 1 = Like, 2 = Dislike
fixation_frame_dislike = fixation_file.loc[fixation_file['block'] == 2] # 1 = Like, 2 = Dislike


fixation_frame_like = fixation_frame_like.drop(['block'], 1)
fixation_frame_like = fixation_frame_like.drop(['name'], 1)
fixation_frame_like = fixation_frame_like.drop(['rt'], 1)

fixation_frame_dislike = fixation_frame_dislike.drop(['block'], 1)
fixation_frame_dislike = fixation_frame_dislike.drop(['name'], 1)
fixation_frame_dislike = fixation_frame_dislike.drop(['rt'], 1)

fixation_frame_like.to_csv("Output/fixations_FFA_Like_v1.csv",index = False)
fixation_frame_dislike.to_csv("Output/fixations_FFA_Dislike_v1.csv",index = False)



## ####################   END  ################ #

In [111]:
fixation_file = fixation_file.drop(['name'], 1)
fixation_file = fixation_file.drop(['rt'], 1)
fixation_file.to_csv("Output/fixations_FFA_v3.csv", index = False)
fixation_file

,parcode,trial,fix_item,fix_time,block
0,0,0,0,437,2.0
1,0,0,0,164,2.0
2,0,0,1,302,2.0
3,0,0,1,198,2.0
4,0,0,1,378,2.0
5,0,0,2,432,2.0
6,0,0,2,262,2.0
7,0,0,1,358,2.0
8,0,0,1,358,2.0
9,0,0,1,438,2.0


In [ ]:
for i in range(len(participant)):
    trial_n = 240
    for  j in range(trial_n):
        fixat_data_trial=fixat_data.loc[(fixat_data['DATA_FILE']==participant[i]) & (fixat_data['trial']== j+1) & (fixat_data['trial_type']== 'Experiment')]
        fixat_n = len(fixat_data_trial)
        for  k in range(fixat_n):
            



In [147]:
# From BlinkMatrix (from FoodFrameAnalysis.ipnyb) we find that we have to take out participants

#Part2Delete = ['p9.edf' , 'p10.edf' , 'p20.edf' , 'p28.edf', 'p33.edf' , 'p36.edf', 'p40.edf']
Part2Delete = ['p9' , 'p10' , 'p20' , 'p28', 'p33' , 'p36', 'p40']



delete_index = []
delete_list = []

for j in range(len(Part2Delete)):
    for i in range(len(participant)):
        if participant_names[i]==Part2Delete[j]:
            #delete_index.append(Part2Delete)
            delete_index=i 
            print('Eliminated participant ', delete_index+1) # participant number (+1 since we start with number 0 in the vector)
            print('Eliminated participant name', participant[delete_index])
            delete_list.append(delete_index)
 

for i in range(len(participant)):
    participant_names = np.delete(participant_names, [delete_list], None)
    participant = np.delete(participant, [delete_list], None)
            

Eliminated participant  6
Eliminated participant name p11.edf
Eliminated participant  7
Eliminated participant name p12.edf
Eliminated participant  17
Eliminated participant name p23.edf
Eliminated participant  25
Eliminated participant name p32.edf
Eliminated participant  30
Eliminated participant name p39.edf
Eliminated participant  33
Eliminated participant name p43.edf


In [26]:
len(participant)

39

In [27]:
len(participant_names)

39

In [14]:
i=0
GSF_matrix = []
#Run for each individual participant
for i in range(len(participant)):
    #Extract the number of trials for this participant
    #pX_indSubj = SACCdata.loc[(SACCdata['DATA_FILE']==participant[i]) & (SACCdata['trial_type']== 'Experiment')]
    #Add number of trials for the experiment
    #trial_n = pX_indSubj['trial'].unique()
    trial_n = range(240)
    for  j in range(len(trial_n)):
            # extract saccade data for this participant
            GSF_data=SACCdata.loc[(SACCdata['DATA_FILE']==participant[i]) & (SACCdata['trial']== j+1) & (SACCdata['trial_type']== 'Experiment')]



'p43'

In [ ]:
   
participant_names = np.delete(participant_names, [delete_index], None)
participant = np.delete(participant, [delete_index], None)
participant

In [6]:
#for i in [6, 7, 17, 25, 30, 33]:

delete_index = []
for i in range(len(participant)):
    if participant_names[i]=='p40':
        delete_index = i

participant_names = np.delete(participant_names, [delete_index], None)
participant = np.delete(participant, [delete_index], None)

AttributeError: 'DataFrame' object has no attribute 'Part'